### Assignment 1

Create a simple assistant that uses any LLM and should be Pydantic, when we ask about any product it should give you three parts of information. Str(productName), Str(ProductDetails), Int / Float (priceUSD). Use the ChatPrompTemplate

In [9]:
# Putting in all the env variables in the project

import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Assignment_1"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [14]:
# Initializing the Model. 

from langchain_groq import ChatGroq
model = ChatGroq(model="qwen-qwq-32b", temperature=0.2)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x123813ed0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1238b0910>, model_name='qwen-qwq-32b', temperature=0.2, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Defining the output schema
class Product(BaseModel):
    productName: str = Field(description="Name of the Product")
    productDetails: str = Field(description="Details of the Product")
    priceUSD: float = Field(gt=0)


# Adding a query intented to prompt a language model to populate the data structure.
product_query = "Tell me about the PlayStation 5"

# Setting up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Product)

# Setting up the prompt
prompt = ChatPromptTemplate(
    [
        ("system","You are ecommerce website where you share the product name, details and price in USD you are asked for in JSON Format"),
        ("human","{format_instructions} \n {product_query}")
    ]
)

# Create the chain
chain  = prompt | model | parser


# Invoke the chain
result = chain.invoke(
    {
        "product_query":product_query,
        "format_instructions":parser.get_format_instructions()
    }
)

print(result)


{'productName': 'PlayStation 5', 'productDetails': 'The PlayStation 5 is a next-generation gaming console featuring a custom AMD Zen 2 processor, AMD RDNA 2 GPU, and a ultra-fast custom SSD for lightning-fast load times. It supports 4K resolution, ray tracing, and offers immersive gaming experiences with backward compatibility for many PlayStation 4 games. Includes the DualSense wireless controller and HD Tempest 3D Audio Technologies for enhanced audio immersion. The console also comes with an HD camera and microphone for social interactions and streaming.', 'priceUSD': 499}
